# Household Income by ZIP code in NY
By Paige DeFiori

This notebook will unpack data from the ([table B19001](https://censusreporter.org/data/table/?table=B19001&geo_ids=860|04000US36)) on household income 2015-2019 in NY, further analyzed by ZIP code. The purpose is to see which areas in the city have the most levels of household income, which could realte to our final project as socioeconomic status is a factor for the levels of impacts COVID-19 has on different individuals, states and/or countries. This of course is based off of census data, therefore is just used to analyze one state while my parnter analyzes income levels LA. *Since there are so many more areas in the state of NY than the city of LA (which my partner is using), I will eventually reduce the scope of the income brackets to be below \\$50,000 dollars and above \\$150,000 dollars to minimize data comparison, as well as have more drastic comparison to the US average household income of \\$68,703.*

## Establish Libraries

In [ ]:
import pandas as pd

import geopandas as gpd

import contextily as ctx

import matplotlib.pyplot as plt

## Importing Data

Importing the data and running a few commands to see the depth of the dataset:

In [ ]:
gdf = gpd.read_file('data/acs2019_5yr_B19001_86000US12120.geojson')

`shape` will show the rows & columns, so I can see the size of the data more clearly

In [ ]:
gdf.shape

In [ ]:
gdf.head()

## Plotting and Visualizing the Data

`.plot()` command to visualize the data. It should show the state of NY sperated by ZIP codes.

In [ ]:
gdf.plot(figsize=(10,10))

The `set_option` command to reveal all of the data, instead of abbreviated columns.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

`.info()` will now view all the data types

In [ ]:
gdf.info()

The columns are pretty unclear based off `.head()`, so now I will rename the columns and cutting the dataset down to the income brackets specified in the intro:

`list()` will show all column names as of now, for all of the data:

In [ ]:
list(gdf)

The cells below remove columns, based on the income cut offs noted in the introduction, as well as rename the columns to be properly labeled.

In [ ]:
columns_to_keep = [
 'geoid', 
 'name', 
 'B19001001',
 'B19001002',
 'B19001003',
 'B19001004',
 'B19001005',
 'B19001006',
 'B19001007',
 'B19001008',
 'B19001009',
 'B19001010',
 'B19001016',
 'B19001017',
 'geometry']

New columns are now assigned as the initial datase name for ease

In [ ]:
gdf = gdf[columns_to_keep]

`.columns` to rename the columns to the household income brackets:

In [ ]:
gdf.columns = ['geoid',
 'ZIP Code',
 'Total',
 '< 10,000',
 '10,000 to 14,999',
 '15,000 to 19,999',
 '20,000 to 24,999',
 '25,000 to 29,999',
 '30,000 to 34,999',
 '35,000 to 39,999',
 '40,000 to 44,999',
 '45,000 to 49,999',
 '150,000 to 199,999',
 '200,000+',
 'geometry']

Use `.head()` to double-check that the columns are now properly renamed to their corresponding income values.

In [ ]:
gdf.head()

## Plotting based on Income Brackets

In [ ]:
gdf['200,000+'].plot.hist(bins=15)

In [ ]:
gdf['< 10,000'].plot.hist(bins=15)

These two histograms reveals that `<$10,000` frequency distribution is similar to that of the frequency of households with `>$200,000` income. Which shows they are both pretty extreme and possibly outliers.

`.sorted` in `decending` order will show me which ZIP code has the most households living in it, regardless of income bracket.

In [ ]:
gdf_sorted = gdf.sort_values(by='Total',ascending = False)

In [ ]:
gdf_sorted[['geoid','Total']].head(10)

Now plotting these top 10 most populated ZIP codes will reveal where they are in NY.

In [ ]:
gdf_sorted.head(10).plot()

Now making it more colorized and specific:

In [ ]:
gdf_sorted.head(10).plot(figsize=(10,10),column='Total',legend=True)

The next processs is to delete the data that has 0 households in the `Total` column, as these are irrelevant.

In [ ]:
gdf[gdf['Total']==0]

Listed above, is the total ZIP codes with 0 households accounted for in 2015-2019. 
Below will remove thes ZIP codes from the main dataset:

In [ ]:
gdf = gdf[gdf['Total']>0]

In [ ]:
gdf[['geoid','Total']]

The average US household income is $68,703. Below, I add columns by incomes `> the US average` and incomes `< the US average`.

In [ ]:
below_avg = gdf['< 10,000'] + gdf['10,000 to 14,999'] + gdf['15,000 to 19,999'] + gdf['20,000 to 24,999'] + gdf['25,000 to 29,999'] + gdf['30,000 to 34,999'] + gdf['35,000 to 39,999'] + gdf['40,000 to 44,999'] + gdf['45,000 to 49,999']
above_avg = gdf['150,000 to 199,999'] + gdf['200,000+']

In [ ]:
gdf['% below US Avg'] = below_avg/gdf['Total']*100
gdf['% above US Avg'] = above_avg/gdf['Total']*100

Now, there should be two columns added to the dataset for above and below the US avg, written as percentages for each ZIP code in the US.

In [ ]:
gdf.head()

## Now to more directly visualize the differences in incomes:

In [ ]:
# create the 1x2 subplots
fig, axs = plt.subplots(1, 2, figsize=(25, 25))

# name each subplot
ax1, ax2 = axs

# regular count map on the left
gdf.plot(column='% below US Avg', 
            cmap='CMRmap', 
            scheme='equal_interval',
            k=5, 
            edgecolor='white', 
            linewidth=0, 
            alpha=0.75, 
            ax=ax1, # this assigns the map to the subplot,
            legend=True
           )

ax1.axis("off")
ax1.set_title("Percent of Household Incomes Below US Avg")
leg = ax1.get_legend()
leg.set_bbox_to_anchor((0., 1, 0.2, 0.))
ax1.get_legend().set_title('percent') 

# spatial lag map on the right
gdf.plot(column='% above US Avg', 
            cmap='CMRmap', 
            scheme='equal_interval',
            k=5, 
            edgecolor='white', 
            linewidth=0, 
            alpha=0.80, 
            ax=ax2, # this assigns the map to the subplot
            legend=True
           )
ax2.axis("off")
ax2.set_title("Percent of Household Incomes Above US Avg")
leg = ax2.get_legend()
leg.set_bbox_to_anchor((0., 1, 0.2, 0.))
ax2.get_legend().set_title('percent') 

The comparison above shows that the most affluent households in NY live in areas near Westchester and Manhattan, whereas the below avg household income are located near Queens and Nassau. Most of the state's households rests in the below US avg category. Which can be compared to cases of death by COVID-19 to find a correlation, aided by [this Policio article](https://www.politico.com/states/new-york/city-hall/story/2020/05/18/poorest-nyc-neighborhoods-have-highest-death-rates-from-coronavirus-1284519).

Just out of personal interest, and to confirm the areas specifed above, the plot below is the Top 500 ZIP codes in NY with household incomes \\$150,000 and above:

In [ ]:
# create the 1x2 subplots
fig, axs = plt.subplots(1, figsize=(11, 11))

# name each subplot
ax1 = axs
gdf_new_sorted = gdf.sort_values(by='% above US Avg', ascending = False)
gdf_new_sorted.head(500).plot(figsize=(10,10),
                              column='Total',
                              cmap='CMRmap', 
                              scheme='equal_interval',
                              k=5,
                              edgecolor='white', 
                              linewidth=0, 
                              alpha=0.75, 
                              ax=ax1, 
                              legend=True)

ax1.set_title("Top 500 NY ZIP Codes with Household Incomes Above US Avg, in %")
ax1.axis("off")
leg = ax1.get_legend()
ax1.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))
ax1.get_legend().set_title('percent') 